<a href="https://colab.research.google.com/github/de4aultname/ml-seminars-2019/blob/master/1704%20(NN).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NN

Приближаем искомую зависимость композицией функций особого класса (aka нейронами). Способ аппроксимации --- минимизация функции потерь. В случае, когда в композиции один нейрон, это выглядит так: $$\frac{1}{n} \sum_{i = 1}^{n} \mathcal{L}(a, x^i, y_i) \rightarrow \underset{w}{\min},$$

где функция особого класса выглядит как $a(x^i, w) = \sigma \left (\sum_{j = 1}^p \omega_j x_j^i - \omega_0 \right )$.

Здесь $x^i = (x_1^i, \dots, x_p^i)$ и $y_i$ --- реализации $\bar{\eta}$ и $\xi$ соответственно, функция $\sigma(x)$ --- непостоянная и монотонно возрастающая непрерывная функция, "функция активации".

Примеры функций активаций:

* Сигмоидная функция: $\sigma(z) = \frac{1}{1+e^{-a\,z}}$, $a \in \mathbb{R}$;	
* Softmax: $SM_i(z)  = \frac{e^{z_i}}{\sum_{k=1}^{K}e^{z_k}}$;
* Гиперболический тангенс: $\sigma(z) = \frac{e^{a\,z} - e^{-a\,z}}{e^{a\,z} + e^{-a\,z}}$, $a \in \mathbb{R}$;
* Выпрямитель: $ReLU(p) = \max (0,p)$.

Как оптимизируем: SGD (или любой другой приглянувшийся оптимизационный алгоритм), для вычисления градиента используется алгоритм backpropagation (что на самом деле есть грамотное использование chain rule для дифференцирования сложной функции)


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
from torch.utils.data.sampler import SubsetRandomSampler, Sampler

train_loader = torch.utils.data.DataLoader(datasets.MNIST('../mnist_data', 
                                                          download=True, 
                                                          train=True,
                                                          transform=transforms.Compose([
                                                              transforms.ToTensor() 
                                                              #transforms.Normalize((0.1307,), (0.3081,)) 
                                                          ])), 
                                           batch_size=10, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(datasets.MNIST('../mnist_data', 
                                                          download=True, 
                                                          train=False,
                                                          transform=transforms.Compose([
                                                              transforms.ToTensor()
                                                              #transforms.Normalize((0.1307,), (0.3081,)) 
                                                          ])), 
                                           batch_size=10, 
                                           shuffle=True)

Extracting ../mnist_data/MNIST/raw/train-images-idx3-ubyte.gz to ../mnist_data/MNIST/raw


Extracting ../mnist_data/MNIST/raw/train-labels-idx1-ubyte.gz to ../mnist_data/MNIST/raw


Extracting ../mnist_data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../mnist_data/MNIST/raw


Extracting ../mnist_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../mnist_data/MNIST/raw
Processing...
Done!


In [3]:
class Flattener(nn.Module):
    def forward(self, x):
        batch_size, *_ = x.shape
        return x.view(batch_size, -1)

def train_model(model, train_loader, val_loader, loss, optimizer, num_epochs):    
    loss_history = []
    train_history = []
    val_history = []
    for epoch in range(num_epochs):
        model.train() 
        loss_accum = 0
        correct_samples = 0
        total_samples = 0
        for i_step, (x, y) in enumerate(train_loader):
            prediction = model(x)    
            loss_value = loss(prediction, y)
            optimizer.zero_grad()
            loss_value.backward()
            optimizer.step()
            
            _, indices = torch.max(prediction, 1)
            correct_samples += torch.sum(indices == y)
            total_samples += y.shape[0]
            
            loss_accum += loss_value

        ave_loss = loss_accum / (i_step + 1)
        train_accuracy = float(correct_samples) / total_samples
        val_accuracy = compute_accuracy(model, val_loader)
        
        loss_history.append(float(ave_loss))
        train_history.append(train_accuracy)
        val_history.append(val_accuracy)
        
        print("Average loss: %f, Train accuracy: %f, Val accuracy: %f" % (ave_loss, train_accuracy, val_accuracy))
        
    return loss_history, train_history, val_history
        
def compute_accuracy(model, loader):
    model.eval() 
    correct_samples = 0
    total_samples = 0
    for x, y in loader:
        total_samples += y.shape[0]
        prediction = torch.argmax(model(x), dim=1)
        correct_samples += torch.sum(prediction == y)
    return float(correct_samples)/total_samples    

## gridsearch

Про выбор параметров: в прошлый раз было видно, что SGD не особо сходится на 3-5 итерациях, выберем адаптивный метод оптимизации, поэтому выберем адаптивный метод Adadelta, в силу его адапитивности не имеет смысла перебирать много значений learning rate, т.е. длины шага, который делаем вдоль градиента.

Поскольку данные не стандартизованы, мы теоретически можем переживать за то, чтобы градиенты не стали "exploding". Целевая функция оптимизации здесь не такая, как в LR, поэтому не стоит опираться на тот набор значений, который перебирали там. 
Поэтому здесь с точки зрения коэффициента регуляризации (тут не обратная величина, как в scikit-learn LR) идем в сторону уменьшения.

Про выбор значений размера "скрытого" слоя --- перебираем с шагом 100 от 700 до 100 соответственно, это тоже логично с учетом того, что признаков 784.

In [4]:
def search_param(nn_model, train_loader, val_loader, lr, l2, epochs):
    loss = nn.CrossEntropyLoss().type(torch.FloatTensor)
    optimizer = optim.Adadelta(nn_model.parameters(), lr=lr, weight_decay=l2)
    loss_history, train_history, val_history = train_model(nn_model, train_loader, test_loader, 
                                                           loss, optimizer, epochs)
    return loss_history, train_history, val_history, nn_model


lrs = [1, 1e-1, 1e-2]
regs = [1e-1, 1e-2, 1e-3, 1e-4, 1e-5]
hidden_sizes = [600, 500, 400, 300, 200, 100]

params = {
    'batch size': [],
    'lr': [],
    'l2': [],
    'accuracy': []
}
best_model = None
best_acc = 0
best_lr = 0
best_l2 = 0
for hidden_size in hidden_sizes:
    for lr in lrs:
        for l2 in regs:
            params['lr'].append(lr)
            params['l2'].append(l2)
            model = nn.Sequential(
                Flattener(),
                nn.Linear(784, hidden_size),
                nn.ReLU(inplace=True),
                nn.Linear(hidden_size, 10),
            )
            loss_history, train_history, val_history, model = search_param(model, train_loader, test_loader, lr, l2, 5)
            params['accuracy'].append(val_history[-1])
            print(f'current accuracy {val_history[-1]}')
            if best_acc < val_history[-1]:
                best_acc = val_history[-1]
                best_model = model
                best_lr = lr
                best_l2 = l2
            
nn_model = best_model

Average loss: 0.971600, Train accuracy: 0.752783, Val accuracy: 0.790300
Average loss: 0.911124, Train accuracy: 0.797400, Val accuracy: 0.828200
Average loss: 0.904837, Train accuracy: 0.802067, Val accuracy: 0.815100
Average loss: 0.903968, Train accuracy: 0.803867, Val accuracy: 0.821900
Average loss: 0.900455, Train accuracy: 0.808167, Val accuracy: 0.813200
current accuracy 0.8132
Average loss: 0.410793, Train accuracy: 0.878017, Val accuracy: 0.910500
Average loss: 0.353632, Train accuracy: 0.900167, Val accuracy: 0.904700
Average loss: 0.341428, Train accuracy: 0.905800, Val accuracy: 0.906700
Average loss: 0.338773, Train accuracy: 0.906017, Val accuracy: 0.913800
Average loss: 0.336325, Train accuracy: 0.908483, Val accuracy: 0.913500
current accuracy 0.9135
Average loss: 0.231880, Train accuracy: 0.930583, Val accuracy: 0.943700
Average loss: 0.169628, Train accuracy: 0.949233, Val accuracy: 0.954000
Average loss: 0.161541, Train accuracy: 0.951450, Val accuracy: 0.962900
Ave

In [5]:
nn_model

Sequential(
  (0): Flattener()
  (1): Linear(in_features=784, out_features=600, bias=True)
  (2): ReLU(inplace=True)
  (3): Linear(in_features=600, out_features=10, bias=True)
)

In [9]:
print("Best lr: %2.4f" % best_lr)
print("Best l2 regularization parameter: %2.6f" % best_l2)

Best lr: 1.0000
Best l2 regularization parameter: 0.000010


In [7]:
test_accuracy = compute_accuracy(nn_model, test_loader)
print("Test accuracy: %2.4f" % test_accuracy)

Test accuracy: 0.9806
